In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import shutil


In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to prevent TensorFlow from pre-allocating all GPU memory
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is configured successfully!")
    except RuntimeError as e:
        print(e)


In [ ]:
# Define paths
data_dir = "/kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)"  # Update this to your dataset path
drowsy_dir = os.path.join(data_dir, 'Drowsy')
non_drowsy_dir = os.path.join(data_dir, 'Non Drowsy')

# Ensure dataset directories exist
assert os.path.exists(drowsy_dir), "Drowsy folder not found"
assert os.path.exists(non_drowsy_dir), "Non-drowsy folder not found"

# Create train, val, test directories
output_dir = "prepared_data"
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")
test_dir = os.path.join(output_dir, "test")

for split in [train_dir, val_dir, test_dir]:
    for category in ["drowsy", "non_drowsy"]:
        os.makedirs(os.path.join(split, category), exist_ok=True)


In [ ]:
# Helper function to split and copy files
def split_and_copy(source_dir, target_dirs, split_ratios):
    filenames = os.listdir(source_dir)
    train, test = train_test_split(filenames, test_size=split_ratios[1] + split_ratios[2], random_state=42)
    val, test = train_test_split(test, test_size=split_ratios[2] / (split_ratios[1] + split_ratios[2]), random_state=42)
    
    splits = {"train": train, "val": val, "test": test}
    for split_name, files in splits.items():
        split_path = target_dirs[split_name]
        category_name = os.path.basename(source_dir)  # 'Drowsy' or 'Non Drowsy'
        category_path = os.path.join(split_path, category_name.lower().replace(" ", "_"))  # Convert to lowercase and underscores
        
        # Create subdirectory if it doesn't exist
        os.makedirs(category_path, exist_ok=True)
        
        for file in files:
            shutil.copy(os.path.join(source_dir, file), os.path.join(category_path, file))

In [ ]:
split_and_copy(drowsy_dir, {"train": train_dir, "val": val_dir, "test": test_dir}, [0.7, 0.2, 0.1])
split_and_copy(non_drowsy_dir, {"train": train_dir, "val": val_dir, "test": test_dir}, [0.7, 0.2, 0.1])


In [ ]:
# ImageDataGenerator for data loading and augmentation
train_datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range=20, zoom_range=0.15, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:

class ClassToken(layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        
    def build(self, input_shape):
        self.class_token = self.add_weight(
            shape=(1, 1, self.d_model),
            initializer='zeros',
            trainable=True,
            name='class_token'
        )
        
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        class_token = tf.tile(self.class_token, [batch_size, 1, 1])
        return tf.concat([class_token, inputs], axis=1)

In [ ]:
class PositionEmbedding(layers.Layer):
    def __init__(self, num_patches, d_model):
        super().__init__()
        self.num_patches = num_patches
        self.d_model = d_model
        
    def build(self, input_shape):
        self.position_embeddings = self.add_weight(
            shape=(1, self.num_patches + 1, self.d_model),
            initializer='zeros',
            trainable=True,
            name='position_embeddings'
        )
        
    def call(self, inputs):
        return inputs + self.position_embeddings

In [ ]:
def create_vit_model(input_shape=(224, 224, 3), num_classes=2, 
                    d_model=64, num_heads=4, num_layers=4, 
                    mlp_dim=128, patch_size=16):
    num_patches = (input_shape[0] // patch_size) ** 2  
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(d_model, patch_size, strides=patch_size, name='patch_embedding')(inputs)
    x = layers.Reshape((num_patches, d_model))(x)
    
    x = ClassToken(d_model)(x)
    
    x = PositionEmbedding(num_patches, d_model)(x)
    
    for _ in range(num_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=d_model//num_heads
        )(x1, x1)
        
        x = layers.Add()([x, attention_output])
        
        x2 = layers.LayerNormalization(epsilon=1e-6)(x)
        
        mlp_output = layers.Dense(mlp_dim, activation='gelu')(x2)
        mlp_output = layers.Dense(d_model)(mlp_output)
        
        x = layers.Add()([x, mlp_output])
    
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = x[:, 0, :]  
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs, outputs)

In [ ]:

vit_model = create_vit_model()


In [ ]:
vit_model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [ ]:
history = vit_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


In [ ]:
vit_model.save("drowsiness_detection_vit_model.h5")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(vit_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Apply default optimizations
tflite_model = converter.convert()

In [ ]:
with open('drowsiness_detection_vit_model_tflite.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
test_generator = val_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
test_loss, test_acc = vit_model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")